# Running the computation of the molecular orbital overlaps and time-overlaps using Libra/CP2K interface in DFT framework on HPCs 

In this tutorial, we will compute the molecular orbital (MO) overlaps and time-overlaps using the Libra/CP2K interface and Libint2 library. Detailed information are given in the [README.md](../README.md) and [../1_desktop/1_example_TiO2/tutorial.ipynb](../1_desktop/tutorial.ipynb) files. In this notebook, we will test it by using only one job and for the pre-computed MD trajectory obtained from the previous step in [here](../../../6_step1_cp2k).


## Table of contents
<a name="toc"></a>
1. [Importing needed libraries](#import)


2. [Overview of required files](#required_files)


3. [Distributing the jobs](#job_dist)


### A. Learning objectives

* To be able to run the molecular orbital overlap calculations on an HPC by distributing the calculations over multiple jobs 

### B. Use cases

* [Distributing the jobs](#job_dist)


### C. Functions

- `libra_py`
  - `CP2K_methods`
    -  [`distribute_cp2k_libint_jobs`](#job_dist)
  

## 1. Importing needed libraries <a name="import"></a>
[Back to TOC](#toc)

Import `CP2K_methods` module using the following commands:

In [1]:
import os
import sys
from libra_py import CP2K_methods

/projects/academic/cyberwksp21/Software/Conda/Miniconda3/envs/libra/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: to-Python converter for std::vector<std::vector<int, std::allocator<int> >, std::allocator<std::vector<int, std::allocator<int> > > > already registered; second conversion method ignored.
  return f(*args, **kwds)
/projects/academic/cyberwksp21/Software/Conda/Miniconda3/envs/libra/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: to-Python converter for boost::python::detail::container_element<std::vector<std::vector<int, std::allocator<int> >, std::allocator<std::vector<int, std::allocator<int> > > >, unsigned long, boost::python::detail::final_vector_derived_policies<std::vector<std::vector<int, std::allocator<int> >, std::allocator<std::vector<int, std::allocator<int> > > >, false> > already registered; second conversion method ignored.
  return f(*args, **kwds)
/projects/academic/cyberwksp21/Software/Conda/Miniconda3/envs/libra/lib/python3.6/import

## 2. Overview of required files <a name="required_files"></a>
[Back to TOC](#toc)

The following file is needed to run the calculatoins for computing the MO overlaps.

* `es_diag_temp.inp`

A sample CP2K input file to run the electronic structure calculations. This file can be a copy of the [MD input](../../../6_step1_cp2k/1_DFT/md.inp) but with `RUN_TYPE ENERGY` in the `&GLOBAL` section.

* `run_template.py`

A file that contains the data to run the MO overlap calculations. The details about this file is brought in the [README.md](../../../README.md) file. 

* `submit_template.slm`

A sample file for submitting the job that runs `python run.py`. The `run.py` file is a copy of the `run_template.py` which has the initial and final step based on the number of jobs.

* `../../../../6_step1_cp2k/1_DFT/1_example_TiO2/Rutile_TiO2_MD-pos-1.xyz`

The MD trajectory `.xyz` file obtained from [step1](../../../../6_step1_cp2k/1_DFT/1_example_TiO2/tutorial.ipynb). CP2K stores the MD trajectory data in `*-pos-*.xyz` files.


# 3. Computing the overlap calculations <a name="job_dist"></a>
[Back to TOC](#toc)

To run the calculations you need to specify the initial and final step of the trajectory and the number of jobs. Libra will split the trajectory based on these values and will submit them by creating the specific folders for each job. The parameters in this file are as follows:

`run_slurm`: If set to `True`, it will use the slurm environment to submit the jobs using the `submit_template` file. If it is set to `False`, it will run the calculations on the active session but multiple jobs will be run on the current active session. Therefore, if you run the calculations with `run_slurm = False`, set `njobs` to `1`.

`istep`: The initial step of the trajectory.

`fstep`: The final step of the trajectory.

`njobs`: Number of jobs.

`submission_exe`: The submission executable

Then the function `CP2K_methods.distribute_cp2k_libint_jobs` will distribute and submit the jobs.



In [2]:
run_slurm = True
submit_template = 'submit_template.slm'
run_python_file = 'run_template.py'
istep = 1200
fstep = 1225
njobs = 5
submission_exe = 'sbatch'
# Removing the previous folders if existed. You can keep them as well 
# but Libra will overwrite some of the data if their names are the same
os.system('rm -rf res job* all_logfiles all_pdosfiles')

print('Distributing jobs...')
CP2K_methods.distribute_cp2k_libint_jobs(submit_template, run_python_file, istep, fstep, njobs, run_slurm, submission_exe)

Distributing jobs...
Submitting job 1
Job 0 istep 1200 fstep 1206 nsteps 6
Submitted job 0
Submitting job 2
Job 1 istep 1205 fstep 1211 nsteps 6
Submitted job 1
Submitting job 3
Job 2 istep 1210 fstep 1216 nsteps 6
Submitted job 2
Submitting job 4
Job 3 istep 1215 fstep 1221 nsteps 6
Submitted job 3
Submitting job 5
Job 4 istep 1220 fstep 1225 nsteps 5
Submitted job 4
